In [15]:
from scipy.io import loadmat
import numpy as np
import pickle

In [16]:
file = loadmat('data-starplus-04847-v7.mat')

In [17]:
file.keys()

dict_keys(['__header__', '__version__', '__globals__', 'info', 'data', 'meta'])

In [18]:
# read data
raw = file['data']
raw

array([[array([[ 1.33426143,  0.05596435,  7.00415539, ...,  6.11591804,
        -0.68593679, -3.93790586],
       [ 1.33748724,  0.05897628,  7.00766386, ...,  5.45987205,
         1.37000129,  0.36822851],
       [ 1.33477119,  0.05614751,  7.00442137, ...,  6.31717984,
         0.33012149,  0.14191474],
       ...,
       [-0.26159329,  2.40558504, -0.37434724, ...,  4.05418949,
         1.31836904, -0.59829245],
       [-0.46012829, -1.81403296, -0.21181782, ...,  7.31920192,
         1.91321595, -3.23694489],
       [ 1.23531253, -0.97535176,  0.36768722, ...,  3.56770317,
        -1.31750016, -5.49009752]])],
       [array([[ 5.59304672e+00,  2.57430762e+00,  5.26724698e-02, ...,
         9.74958479e+00,  7.22559878e+00, -2.37827329e+00],
       [-2.17269380e+00, -1.69963373e+00, -5.07461655e+00, ...,
         1.35320159e+01, -6.36360383e-01, -5.46173752e+00],
       [ 3.57320834e+00, -1.18878528e+00,  5.47068295e+00, ...,
         5.00685239e+00,  5.59334736e-01, -6.72921025e-02

In [19]:
print(np.shape(raw))
print(np.shape(raw[0]))
print(np.shape(raw[0][0]))
print(np.shape(raw[1,0]))
print(np.shape(raw[2,0]))

(54, 1)
(1,)
(13, 4698)
(55, 4698)
(55, 4698)


In [20]:
coordinate = file['meta']['colToCoord'][0, 0].astype('float64')  #coordinate
print(coordinate.shape)
coordinate

(4698, 3)


array([[24., 25.,  1.],
       [25., 25.,  1.],
       [24., 26.,  1.],
       ...,
       [34., 62.,  8.],
       [35., 62.,  8.],
       [36., 62.,  8.]])

In [21]:
regions = file['meta']['colToROI'][0, 0]    #anatomical regions
regions

array([[array(['RDLPFC'], dtype='<U6')],
       [array(['RDLPFC'], dtype='<U6')],
       [array(['RDLPFC'], dtype='<U6')],
       ...,
       [array(['CALC'], dtype='<U4')],
       [array(['CALC'], dtype='<U4')],
       [array(['CALC'], dtype='<U4')]], dtype=object)

In [22]:
action = file['info']['actionRT'][0]  #action time
print(action)
action[0][0][0]

[array([[0]], dtype=uint8) array([[0]], dtype=uint8)
 array([[2855]], dtype=uint16) array([[0]], dtype=uint8)
 array([[0]], dtype=uint8) array([[2127]], dtype=uint16)
 array([[0]], dtype=uint8) array([[0]], dtype=uint8)
 array([[0]], dtype=uint8) array([[3152]], dtype=uint16)
 array([[2996]], dtype=uint16) array([[0]], dtype=uint8)
 array([[2552]], dtype=uint16) array([[0]], dtype=uint8)
 array([[2497]], dtype=uint16) array([[0]], dtype=uint8)
 array([[0]], dtype=uint8) array([[2872]], dtype=uint16)
 array([[0]], dtype=uint8) array([[0]], dtype=uint8)
 array([[0]], dtype=uint8) array([[0]], dtype=uint8)
 array([[0]], dtype=uint8) array([[0]], dtype=uint8)
 array([[2847]], dtype=uint16) array([[0]], dtype=uint8)
 array([[0]], dtype=uint8) array([[0]], dtype=uint8)
 array([[0]], dtype=uint8) array([[2350]], dtype=uint16)
 array([[2038]], dtype=uint16) array([[3216]], dtype=uint16)
 array([[0]], dtype=uint8) array([[0]], dtype=uint8)
 array([[1359]], dtype=uint16) array([[2640]], dtype=ui

0

In [23]:
tr = 2  #inference for the 3rd trail    P -> S select between 0 to 53
tr

2

In [24]:
totdata = raw[tr, 0]   #total abstracted data
totdata

array([[ -1.94988155,  -0.52482339,  -2.17634407, ...,   0.36915035,
         -3.35687778,  -2.2533888 ],
       [ -0.07264015,   2.46795379,   1.96930878, ...,   0.87795962,
          1.86734881,  -1.87407777],
       [ -5.42586099,  -2.30752888,   2.18383191, ...,   1.14937588,
          0.35025999,  -1.48078613],
       ...,
       [-11.78302729,  -5.48332175,  -2.26232685, ...,   2.19256219,
         -0.22216511,   1.95268804],
       [ -3.56539383,  -2.03520661,  -1.99698161, ...,  -1.813507  ,
         -4.01850018,  -2.64156033],
       [ -9.3105154 ,  -2.19852942,  -1.21166761, ...,   0.42906112,
          0.39785054,  -1.02371639]])

In [25]:
totN = totdata.shape[1]  #total number of voxels
totN

4698

In [26]:
# select only the Region Of Interests(ROI)
roi = ['CALC','LIPL','LT','LTRIA','LOPER','LIPS','LDLPFC']
voi = [v for v in range(totN) if regions[v] in roi] #voxels of interests
np.shape(voi)

C:\Users\rajat\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until


(1713,)

In [27]:
# set parameters
press = ((action[tr][0][0] > 0)*(action[tr][0][0]/1000.0+8.0) + (action[tr][0][0] == 0)*(4.0+8.0))# second stimulus on the screen
data = raw[tr, 0][:, voi]   #abstracted data
N = data.shape[1]  #number of voxels
tp = data.shape[0]   #number of time points
print(N)

1713


In [28]:
# set parameters
press = ((action[tr][0][0] > 0)*(action[tr][0][0]/1000.0+8.0) + (action[tr][0][0] == 0)*(4.0+8.0))# second stimulus on the screen
data = raw[tr, 0][:, voi]   #abstracted data
N = data.shape[1]  #number of voxels
tp = data.shape[0]   #number of time points
print(N)
print(tp)

1713
55


In [29]:
# neighborhood structure ####
def neighbor(N, coord):
    """
    Defines the neighborhood structure
    """

    output = {} # dict for neighbor
#     int v, k
    dis = 0.0
    w = np.zeros((N, N))

    for v in range(0,N):
        for k in range(0,v):
            dis = np.sqrt((coord[v][0]-coord[k][0])**2+(coord[v][1]-coord[k][1])**2+(coord[v][2]-coord[k][2])**2)
            if dis == 0.0:
                w[v, k] = w[k, v] = 0.0
            else:
                w[v, k] = w[k, v] = 1.0/dis

    for v in range(0,N):
        output.update({v: np.where(w[v] == 1.0)[0]})

    return output, w

In [30]:
neigh = neighbor(N, coordinate[voi])
neigh

({0: array([1, 4], dtype=int64),
  1: array([  0,   5, 114], dtype=int64),
  2: array([3], dtype=int64),
  3: array([2, 4, 8], dtype=int64),
  4: array([0, 3, 5, 9], dtype=int64),
  5: array([  1,   4,   6,  10, 119], dtype=int64),
  6: array([  5,   7,  11, 120], dtype=int64),
  7: array([  6,  12, 121], dtype=int64),
  8: array([3, 9], dtype=int64),
  9: array([ 4,  8, 10, 13], dtype=int64),
  10: array([  5,   9,  11,  14, 125], dtype=int64),
  11: array([  6,  10,  12,  15, 126], dtype=int64),
  12: array([  7,  11,  16, 127], dtype=int64),
  13: array([  9,  14,  17, 131], dtype=int64),
  14: array([ 10,  13,  15,  18, 132], dtype=int64),
  15: array([ 11,  14,  16,  19, 133], dtype=int64),
  16: array([ 12,  15,  20, 134], dtype=int64),
  17: array([ 13,  18,  21, 135], dtype=int64),
  18: array([ 14,  17,  19,  22, 136], dtype=int64),
  19: array([ 15,  18,  20,  23, 137], dtype=int64),
  20: array([ 16,  19,  24, 138], dtype=int64),
  21: array([ 17,  22,  26, 139], dtype=int64

In [31]:
neighb = neigh[0] #neighborhood structure
neighb

{0: array([1, 4], dtype=int64),
 1: array([  0,   5, 114], dtype=int64),
 2: array([3], dtype=int64),
 3: array([2, 4, 8], dtype=int64),
 4: array([0, 3, 5, 9], dtype=int64),
 5: array([  1,   4,   6,  10, 119], dtype=int64),
 6: array([  5,   7,  11, 120], dtype=int64),
 7: array([  6,  12, 121], dtype=int64),
 8: array([3, 9], dtype=int64),
 9: array([ 4,  8, 10, 13], dtype=int64),
 10: array([  5,   9,  11,  14, 125], dtype=int64),
 11: array([  6,  10,  12,  15, 126], dtype=int64),
 12: array([  7,  11,  16, 127], dtype=int64),
 13: array([  9,  14,  17, 131], dtype=int64),
 14: array([ 10,  13,  15,  18, 132], dtype=int64),
 15: array([ 11,  14,  16,  19, 133], dtype=int64),
 16: array([ 12,  15,  20, 134], dtype=int64),
 17: array([ 13,  18,  21, 135], dtype=int64),
 18: array([ 14,  17,  19,  22, 136], dtype=int64),
 19: array([ 15,  18,  20,  23, 137], dtype=int64),
 20: array([ 16,  19,  24, 138], dtype=int64),
 21: array([ 17,  22,  26, 139], dtype=int64),
 22: array([ 18,  2

In [32]:
weight = neigh[1] #weights
weight

array([[0.        , 1.        , 0.4472136 , ..., 0.02717632, 0.02722664,
        0.02725696],
       [1.        , 0.        , 0.31622777, ..., 0.02710634, 0.02717632,
        0.02722664],
       [0.4472136 , 0.31622777, 0.        , ..., 0.0280056 , 0.02801659,
        0.0280056 ],
       ...,
       [0.02717632, 0.02710634, 0.0280056 , ..., 0.        , 1.        ,
        0.5       ],
       [0.02722664, 0.02717632, 0.02801659, ..., 1.        , 0.        ,
        1.        ],
       [0.02725696, 0.02722664, 0.0280056 , ..., 0.5       , 1.        ,
        0.        ]])